# Equation of State Class

Initialize an EOS Object to use multiple times.
You can specify an equation of state and if the name is regocnized, the EOS will already have all of it's properties built in.  If not, you can specify the properties.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py

In [12]:
rho_s = 1665.3  #  Central density (density at r = 0)
hc = 197.327           # Conversion factor in MeV fm
G = hc * 6.67259e-45   # Gravitational constant
Ms = 1.1157467e60      # Mass of the sun in kg
mn = 938.926           # Mass of neutron in MeV c^-2

def rho_NS(p): 
    rho_s = 1665.3
    """ density from pressure NS. """
    n = (p*rho_s/363.44)**(1./2.54)
    return (236. * n**2.54 + n *mn)/rho_s 
def dp_dr_NS(r,m,p, Classical = False): 
    """ pressure dependence on radius NS. """
    if Classical:                              # classical model
        y = -m*rho_NS(p)/(r**2 + 1e-20)
    else:                                      # relativistic model
        rh = rho_NS(p)                            
        y = -(p+rh)*(p*r**3 + m)/(r**2 - 2*m*r + 1e-20)
    return y
def dm_dr_NS(r,m,p):
    """ Mass dependence on radius (dm/dr)."""
    return rho_NS(p)*r**2

def standard_eos_pressure(C,k,n):
    try:
        return k * C[0]**(1+1/n)
    except:
        return k * C**(1+1/n) # covers case where rho is entered as a single variable.

In [14]:
class EOS: 
    def __init__(self, name = 'polytropic', P= None, dp_dr= None, notes = "None", constants = {}, h = 0.1,k = 0.1,n = 1,nu = 1,gamma_ = 1,A = None):
        """ 
        Initialize an EOS Object to use multiple times.
        If the name is regocnized, the EOS will already have all of it's properties built in.
        """
        self.name = name.lower()
        rho_s = 1665.3
        self.rho_s = 1665.3 #  Central density (density at r = 0)
        self.M0 = (4*3.14159265*(G**3)*rho_s)**(-0.5)
        self.R0 =  G*self.M0
        self.rho = lambda x : rho_NS(x)
        self.dp_dr = lambda r,m,p, flag: dp_dr_NS(r,m,p, flag)
        self.dm_dr = lambda r, m, p: dm_dr_NS(r,m,p)
        self.constants = {"h":(h, "smoothing length (h)"),"k": (k,"equation of state constant (k)"),"n" :(n,'polytropic index (n)'), 'rho_s': (rho_s, "Central density"), 'nu' : (nu, "Viscosity"),"gamma":(gamma_, "Constant ratio of specific heats"),"A":(A,"function of the points specific entropy. (Rasio&Sharpio 1992)")}
        self.lmbda = lambda M,R : 2*k*(1+n)*np.pi**(-3/(2*n)) * (M*gamma(5/2+n)/R**3/gamma(1+n))**(1/n) / R**2  #is external force constant, ~ 2.01
        ## ++++++++++++++++++++++++++ OLD EOS ++++++++++++++++++++++++++++ ##
        if self.name == 'standard':
            self.notes = 'Old EOS. P(rho).'
            self.P = lambda C : standard_eos_pressure(C,k,n)
        # =========================== Adiabatic EOS ============================= #
        elif self.name == 'adiabatic':
            self.notes = 'Adiabatic EOS for the gas. Used in Rasio & Sharpio,1991. P(rho,v).'
            try: 
                self.P = lambda C : C[0]*C[1]*C[2] # Pressure from rho and velocity
            except:
                raise EvaluationError(str(C)+ "Error. " + self.name + " EOS takes 3 variables: C = [rho,gamma,v]")
        # =========================== Polytope EOS ============================= #
        elif self.name == 'polytropic':
            self.notes = 'Polytropic EOS for the gas. Used in Rasio & Sharpio,1992.'
            if A == None:
                raise NotImplementedError("No initial A specified.")
            try: 
                self.P = lambda C : C[2]*C[0]**C[1] # Pressure from rho and velocity
            except:
                raise EvaluationError(str(C)+ "Error. "+ self.name + " EOS takes 3 variables: C = [rho,gamma,A]")
        else:
            raise NotImplementedError("You are trying to use an unkown Equation of State")
            
    def __str__(self):
        return "EOS Object. Type: " + self.name + "\nNotes: " + self.notes + "\n-------\n"+ str(self.constants) +  "\n\n For more information, see the imported EOS file."
    
    def P_eos(self, rho): 
        """ Takes in rho and returns pressure from EOS. """
        func = self.P
        return func(rho)